## Temperature Ramps

In [13]:
#-----Import some essential packages

from __future__ import division
import hoomd
import hoomd.md

#-----Some important variables

t_0 = 5.4;
t_1 = 5.6;
t_2 = 5.8;
t_3 = 6.0;
t_4 = 6.2;
t_5 = 6.4;
t_6 = 6.6;
t_7 = 6.8;
t_8 = 7.0;
p_max  = 1.4;
steps_ramp = 1e5;

#-----Crea un contexto de simulación

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Extrae la configuración del centro de masa de las partículas

system = hoomd.init.read_gsd("T_CM_5.4_P_1.4_ramp.gsd",frame = -1)

#-----Define cada mesógeno en el marco de referencia local de cada centro de masa.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Declara que las moléculas creadas son cuerpos rígidos

rigid.create_bodies();

#-----Define la energía potencial

nl = hoomd.md.nlist.tree();

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist = nl)
lj.set_params(mode='shift')

#-----Define la interacción entre las especies de la simulación

lj.pair_coeff.set('M','M', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('A','A', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('M','A', epsilon = 1.0, sigma = 1.0)

#-----Selecciona un integrador standard

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define el grupo sobre el cual se integraran las ecuaciones de movimiento

mesogens = hoomd.group.rigid_center();

#----Integrate using NPT

temp1 = hoomd.variant.linear_interp(points = [(0,t_0), (steps_ramp, t_1)]);
temp2 = hoomd.variant.linear_interp(points = [(0,t_1), (steps_ramp, t_2)]);
temp3 = hoomd.variant.linear_interp(points = [(0,t_2), (steps_ramp, t_3)]);
temp4 = hoomd.variant.linear_interp(points = [(0,t_3), (steps_ramp, t_4)]);
temp5 = hoomd.variant.linear_interp(points = [(0,t_4), (steps_ramp, t_5)]);
temp6 = hoomd.variant.linear_interp(points = [(0,t_5), (steps_ramp, t_6)]);
temp7 = hoomd.variant.linear_interp(points = [(0,t_6), (steps_ramp, t_7)]);
temp8 = hoomd.variant.linear_interp(points = [(0,t_7), (steps_ramp, t_8)]);

npt = hoomd.md.integrate.npt(group = mesogens, kT = temp1, tau = 5.0, tauP = 0.5, P = p_max);
npt.randomize_velocities(seed = 42)

notice(2): Group "all" created containing 1000 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
-----
You are using tree neighbor lists. Please cite the following:
* M P Howard, J A Anderson, A Nikoubashman, S C Glotzer, and A Z
  Panagiotopoulos. "Efficient neighbor list calculation for molecular simulation
  of colloidal systems using graphics processing units", Computer Physics
  Communications 203 (2016) 45--52
-----
notice(2): Group "rigid_center" created containing 1000 particles


In [14]:
#-----Ramp1 T : 5.4 -> 5.6
#-----Write output and Run the Simulation

log_file = "T_" + str(t_1) + "_P_" + str(p_max) + "_ramp.log"
gsd_file = "T_" + str(t_1) + "_P_" + str(p_max) + "_ramp.gsd" 
meso_gsd_file = "T_CM_" + str(t_1) + "_P_" + str(p_max) + "_ramp.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = mesogens,
               overwrite = True); 

In [15]:
hoomd.run(steps_ramp)
system.box.get_volume()

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9000
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 1352172 / 1452000 | TPS 17.0155 | ETA 01:37:46
Time 00:00:20 | Step 1352364 / 1452000 | TPS 19.1527 | ETA 01:26:42
Time 00:00:30 | Step 1352554 / 1452000 | TPS 18.9974 | ETA 01:27:14
Time 00:00:40 | Step 1352736 / 1452000 | TPS 18.151 | ETA 01:31:08
Time 00:00:50 | Step 1352918 / 1452000 | TPS 18.1613 | ETA 01:30:55
Time 00:01:00 | Step 1353109 / 1452000 | TPS 19.096 | ETA 01:26:18
Time 00:01:10 | Step 1353292 / 1452000 | TPS 18.1775 | ETA 01:30:30
Time 00:01:20 | Step 1353482 / 1452000 | TPS 18.8838 | ETA 01:26:57
Time 00:01:30 | Step 1353673 / 1452000 | TPS 18.9988 | ETA 01:26:15
Time 00:01:40 | Step 1353864 / 1452000 | TPS 18.9294 | ETA 01:26:24
Time 00:01:50 | Step 1354049 / 1452000 | TPS 18.476 | ETA 01:28:21
Time 00:02:0

14044.230597294665

In [16]:
system.get_metadata()

OrderedDict([('box', <hoomd.data.boxdim at 0x11be91c18>),
             ('particles', <hoomd.data.particle_data at 0x11bf76588>),
             ('number_density', 0.6408325424202068),
             ('bonds', <hoomd.data.bond_data at 0x11bf76eb8>),
             ('angles', <hoomd.data.angle_data at 0x11ba130b8>),
             ('dihedrals', <hoomd.data.dihedral_data at 0x11ba13198>),
             ('impropers', <hoomd.data.dihedral_data at 0x11ba13358>),
             ('constraints', <hoomd.data.constraint_data at 0x11ba13438>),
             ('pairs', <hoomd.data.bond_data at 0x11ba134e0>),
             ('timestep', 1452000)])